В ходе данного соревнования, мы перепробовали большое количество алгоритмов ML, также вариантов извлечения фич.
Финальный алгоритм выглядит таким образом:

### Feature extraction
Для извлечения фич использовалась библиотека Surfboard[1], предоставляющая высокоуровневый API для извлечения фич и статистики.

**Извлекались следующие фичи**:

Основная частота (f0), логарифм энергии (logE), спектральный центроид (spectral centroid),
форманты (formants) (1 - 4), громкость, jitters, shimmers, пересечение нуля (zerocrossing) и MFCC

**Используя представленные статистики:**

Среднее, СКО, минимум, максимум, коэффициент асимметрии (skewness),коэффициент островершинности (kurtosis),
первая производная среднего, первая производная СКО, межквартальное размах.

Такой набор фич составлен на основе проченных научных трудов [2,3,4], а также из общих соображений.

Была попытка извлекать не статистики фич, а сами фичи, но это не принесло улучшения результатов,
поскольку большинство фич было откинуто на этапе Feature selection



### Feature selection
Для улучшения скорости обучения и точности классификаторов использовался выбор фич на основе взаимной информации
(Mutual information). Были выбраны 120 столбцов с наивысшими показателями.

### Классификатор
После многочисленных экспериментов было решено использовать стэкинг из 4-ех классификаторов
 * MLP (размер  скрытого слоя - 200)
 * Случайный лес (500 деревьев)
 * Логистическая регрессия (параметр регуляризации - l2, C = 0.01)
 * SVC (С = 0,01)

Эти 4 классификатора показали наилучшие результаты на кросс-валидации, поэтому было решено объединить их в стэк.
Также они рекомендовались в ряде научных работ
В итоге, ансамбль имел точность ~ 0.82 на кросс валидации по 5 блокам.

### Возможные улучшения
Матрица ошибок показывает, что все классификаторы плохо различают 4 и 2 класс.
Если убрать один из них, точность ансамбля приближается к ~0.95. К сожалению, за время соревнования нам так
и не удалось найти такие фичи или классификаторы, которые смогли бы решить данную проблему
